Make the import

In [1]:
import os
import logging
import numpy as np
import mne
import myonset as dbt

Set the path

In [2]:
path_bdf = os.path.join('.','bdf')
path_mrk = os.path.join('.','automatic_detection')

Set trigger id values: to adapt to your experiment

In [3]:
#triggers values used for trial segmentation (e.g., stimulus or fixation cross)
trig_id = {'red_left':42,'red_right':50,\
           'green_left':35,'green_right':38,\
           }

#triggers values used for detected onset, offset and peak events
emg_id = {'onset': 131, 'offset': 132, 'peak': 133}

# list of response triggers, used to remove bursts detected after response 
resp_list =  [98,162] 

# Load the file

Set EMG file name

In [4]:
name_bdf  = 'C1.bdf'

Extract file name and create log file

In [5]:
nameSubj = name_bdf.split('.')[0]
fname = os.path.join(path_bdf,name_bdf)

logName = os.path.join(path_bdf,nameSubj + '.log')
logging.basicConfig(filename=logName, level=logging.INFO) # needed in jupyter notebook
dbt.set_log_file(logName,overwrite=True)
logging.info("Automatic onsets/offsets detection:")

20


Load raw data

In [6]:
raw = mne.io.read_raw_bdf(fname, preload=True,stim_channel = 'Status')

Extracting EDF parameters from C:\Users\Laure_Spieser\Nextcloud\Documents\myonset_public\emg-process\bdf\C1.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 4212735  =      0.000 ...  2057.000 secs...


In [7]:
raw

<RawEDF | C1.bdf, 7 x 4212736 (2057.0 s), ~225.0 MB, data loaded>

Extract events 

In [8]:
mne_events = mne.find_events(raw, shortest_event=1)

Trigger channel has a non-zero initial value of 130816 (consider using initial_event=True to detect this event)
1726 events found
Event IDs: [65314 65315 65318 65322 65330 65343 65378 65407 65442 65472]


Needed when default Status value is not zero.

In [9]:
#mne_events[:,2] = mne_events[:,2]-mne_events[:,1] 
mne_events[:,2] = mne_events[:,2]-mne_events[1,1] 


# Pre-process signal

In [10]:
raw.info['ch_names']

['EXG1', 'EXG2', 'EXG3', 'EXG4', 'Erg1', 'Erg2', 'Status']

EMG bipolar reference 

In [11]:
mne.set_bipolar_reference(raw,anode=['EXG1','EXG3'], cathode=['EXG2','EXG4'],ch_name=['EMG_L','EMG_R'],\
copy=False)

EEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=2, n_times=4212736
    Range : 0 ... 4212735 =      0.000 ...  2057.000 secs
Ready.
Added the following bipolar channels:
EMG_L, EMG_R


<RawEDF | C1.bdf, 5 x 4212736 (2057.0 s), ~160.7 MB, data loaded>

Set channels

In [12]:
raw.pick(['EMG_L','EMG_R','Erg1','Erg2'])
raw.set_channel_types({'EMG_L':'emg',
                       'EMG_R':'emg',
                       'Erg1':'misc',
                       'Erg2':'misc'})

emg_channels_idx = {0: 'EMG_L', 1: 'EMG_R'}
force_channels_idx = {2: 'Erg1', 3:'Erg2'}
channel_names = list(emg_channels_idx.values())+list(force_channels_idx.values()) 

C:\Users\Laure_Spieser\AppData\Local\Temp\ipykernel_15496\3513116623.py:2: RuntimeWarning: The unit for channel(s) Erg1, Erg2 has changed from V to NA.
  raw.set_channel_types({'EMG_L':'emg',


Filter EMG signals

In [13]:
raw = dbt.use_mne.apply_filter(raw, ch_names = ['EMG_L','EMG_R'],low_cutoff = 10)
logging.info("\tHigh pass filtering of EMG traces at 10Hz")

Filter force signals (not necessary all the time)

In [14]:
raw = dbt.use_mne.apply_filter(raw, ch_names = ['Erg1','Erg2'],high_cutoff = 40)  
logging.info("\tLow pass filtering of EMG traces at 40Hz")

Get pre-processed raw data

In [15]:
data_raw = raw.get_data(['EMG_L','EMG_R','Erg1','Erg2'])

# Segment based on events

Put mne events in our events structure

In [16]:
events = dbt.Events(sample=mne_events[:,0], code=mne_events[:,2], chan=[-1]*mne_events.shape[0],\
                    sf=raw.info['sfreq'])

Define segmentation window

In [17]:
tmin = -.5
tmax = 1.5
epoch_time = dbt.times(tmin,tmax,events.sf)
t0 = dbt.find_times(0,epoch_time)
tmax_sample = dbt.find_times(tmax,epoch_time)

Segment and extract data epochs

In [18]:
events.sf

2048.0

In [19]:
epochs_events = events.segment(code_t0=list(trig_id.values()), tmin=tmin, tmax=tmax)
data_epochs = epochs_events.get_data(data_raw)

Found 512 epoch(s).


# Run automatic detection

Set threshold used for raw and Teager-Kaiser EMG var_onset detection

In [20]:
thEMG_raw = 8 
thEMG_tk = 10 
th_force = 7

logging.info("\t\t threshold for EMG left: " + str(thEMG_raw))
logging.info("\t\t threshold for EMG right: " + str(thEMG_raw))        

logging.info("\t\t threshold for EMG left Teager-Kaiser: " + str(thEMG_tk))
logging.info("\t\t threshold for EMG right Teager-Kaiser: " + str(thEMG_tk))

logging.info("\t\t threshold for force left: " + str(th_force))
logging.info("\t\t threshold for force right: " + str(th_force))        

Compute global variance, this will not be used most of the time

In [21]:
mBlRaw,stBlRaw = dbt.global_var(data_epochs,epoch_time,cor_var = 2.5,use_tkeo = False)
mBlTk,stBlTk = dbt.global_var(data_epochs,epoch_time,cor_var = 2.5,use_tkeo = True)

Big loop doing the detection

In [22]:
for e in range(epochs_events.nb_trials()):
    # Onset and offset EMG detection
    for c in emg_channels_idx.keys():
        
        current = data_epochs[e,c,:]
        
        #Lcal mBl and stBl are recommended, to use global values, use mBlRaw/mBlTk[c] and stBlRaw/stBlTk[c] computed above
        onsets,offsets = dbt.get_onsets(current, epoch_time, sf=events.sf,\
                                        th_raw= thEMG_raw, use_raw=True, time_limit_raw=.025, min_samples_raw=5,\
                                        varying_min_raw=1, mbsl_raw=None, stbsl_raw=None, \
                                        th_tkeo= thEMG_tk, use_tkeo=True, time_limit_tkeo=.025,  min_samples_tkeo=5,\
                                        varying_min_tkeo=0, mbsl_tkeo=None, stbsl_tkeo=None)
        
        # Remove burst starting and ending before time 0
        onsets = [onsets[b] for b in range(len(onsets)) if (offsets[b] > t0)]
        offsets = [offsets[b] for b in range(len(offsets)) if (offsets[b] > t0)]
        # If one onset remains before t0, put its latency to time 0
        onsets = [np.max((b,t0+1)) for b in onsets]
        
        # Remove bursts starting after the first response
        stim = epochs_events.list_evts_trials[e].find_events(code=list(trig_id.values()))
        resp = epochs_events.list_evts_trials[e].find_events(code=resp_list)
        if len(resp) > 0:
            #latency of the first response after the first stimulus
            resp_latency =  epochs_events.list_evts_trials[e].lat.sample[resp[resp > stim[0]][0]]
        else: 
            resp_latency = tmax_sample # if no response, resp latency is equal to tmax
        offsets = [offsets[b] for b in range(len(offsets)) if (onsets[b] < resp_latency)]
        onsets = [onsets[b] for b in range(len(onsets)) if (onsets[b] < resp_latency)]

        # Put in event structure
        onsets_events = dbt.Events(sample=onsets, time=epoch_time[onsets],\
                                   code=[emg_id['onset']]*len(onsets), chan=[c]*len(onsets), sf=epochs_events.sf) 
        offsets_events = dbt.Events(sample=offsets, time=epoch_time[offsets],\
                                    code=[emg_id['offset']]*len(offsets), chan=[c]*len(offsets), sf=epochs_events.sf) 
        
        # Add in epochs events
        epochs_events.list_evts_trials[e].add_events(onsets_events)
        epochs_events.list_evts_trials[e].add_events(offsets_events)
        
    # Onset and offset force detection
    for c in force_channels_idx.keys():

        current_force = data_epochs[e,c,:]
        force_intervals = dbt.detector_var(current_force, epoch_time, th=th_force, time_limit=.050,\
                                           sf=epochs_events.sf, min_samples=15, varying_min=0,\
                                           use_derivative=True, moving_avg_size=10)

        onsets_force = force_intervals[:,0]
        offsets_force = force_intervals[:,1]
           
        # Remove intervals starting and ending before time 0
        onsets_force = [onsets_force[b] for b in range(len(onsets_force)) if (offsets_force[b] > t0)]
        offsets_force = [offsets_force[b] for b in range(len(offsets_force)) if (offsets_force[b] > t0)]
        # If one onset remains before t0, put its latency to time 0
        onsets_force = [np.max((b,t0+1)) for b in onsets_force]
        
        # Remove intervals starting after the first response
        offsets_force = [offsets_force[b] for b in range(len(offsets_force)) if (onsets_force[b] < resp_latency)]
        onsets_force = [onsets_force[b] for b in range(len(onsets_force)) if (onsets_force[b] < resp_latency)]

        # Put in event structure
        onsets_force_events = dbt.Events(sample=onsets_force, time=epoch_time[onsets_force], code=[emg_id['onset']]*len(onsets_force), chan=[c]*len(onsets_force), sf=epochs_events.sf) 
        offsets_force_events = dbt.Events(sample=offsets_force, time=epoch_time[offsets_force], code=[emg_id['offset']]*len(offsets_force), chan=[c]*len(offsets_force), sf=epochs_events.sf) 

        # Get force peaks
        list_signals = dbt.get_signal_portions(current_force, onsets_force_events.lat.sample, offsets_force_events.lat.sample)
        _,peak_sample = dbt.get_signal_max(list_signals)
        peak_sample += onsets_force_events.lat.sample
    
        # Put in event structure
        if len(peak_sample) > 0:
            peaks_force_events = dbt.Events(sample=peak_sample, time=epoch_time[peak_sample], code=[emg_id['peak']]*len(peak_sample) , chan=[c]*len(peak_sample), sf=epochs_events.sf)
        
            # Add in epochs events
            epochs_events.list_evts_trials[e].add_events(onsets_force_events)
            epochs_events.list_evts_trials[e].add_events(offsets_force_events)
            epochs_events.list_evts_trials[e].add_events(peaks_force_events)


# Save in new marker file

First put epoch events in continuous time

In [23]:
continuous_events_with_detection = epochs_events.as_continuous(drop_duplic=True)[0]

Checking for duplicates in events...
0 event(s) removed.


Add those to the original events

In [24]:
events.add_events(continuous_events_with_detection, drop_duplic=True)

Checking for duplicates in events...
1198 event(s) removed.


Save in folder path_mrk (defined above)

In [25]:
fname_detected_mrk = os.path.join(path_mrk, nameSubj+'_detect_emg.csv')
events.to_csv(fname_detected_mrk,\
              header='sample,time,code,chan', sep=',',\
              save_sample=True, save_time=True, save_code=True, save_chan=True)

logging.info("\tEvents saved in " + fname_detected_mrk) 
